In [1]:
import pandas as pd
import geopandas as gpd
import re
import matplotlib as plt
import numpy as np

In [2]:
trash = pd.read_csv('cleaned_trash_haulers.csv')

In [5]:
trash['Zip Code'] = trash['Zip Code'].astype('str')
trash['Zip Code'] = trash['Zip Code'].str.extract(r'(\d{5})')

# Red River Waste Solutions Total Damages
- For each address:
    - `>= 3 missed pickups on 3 unique dates`
    - `Within a 6 month timespan`
    + `+ $1500`

In [6]:
trash

,Request Number,Date Opened,Request,Description,Zip Code,Trash Hauler,Trash Route,LONGITUDE,LATITUDE,Clean Address,Missed
0,25270,2017-11-01,Trash - Backdoor,"HOUSE WITH THE WHEEL CHAIR RAMP, THEY SHARE DR...",37207,RED RIVER,3205,-86.815392,36.217292,3817 CROUCH DR,False
1,25274,2017-11-01,Trash - Curbside/Alley Missed Pickup,CURB/TRASH MISS TUESDAY.,37218,RED RIVER,4202,-86.838103,36.213470,4028 CLARKSVILLE PIKE,True
2,25276,2017-11-01,Trash - Curbside/Alley Missed Pickup,CURB/TRASH MISS TUESDAY.,37209,RED RIVER,4205,-86.885562,36.142923,6528 THUNDERBIRD DR,True
3,25307,2017-11-01,Trash - Curbside/Alley Missed Pickup,MISSED,37207,WASTE IND,2206,-86.789170,36.212652,2603 OLD MATTHEWS RD,True
4,25312,2017-11-01,Trash - Curbside/Alley Missed Pickup,MISSED THE EVEN SIDE OF THE ROAD.,37209,RED RIVER,4203,-86.874995,36.154861,604 CROLEY DR,True
...,...,...,...,...,...,...,...,...,...,...,...
20221,267125,2019-11-01,Trash - Curbside/Alley Missed Pickup,MISSED...NEIGHBORS MISSED,37013,RED RIVER,4502,-86.633970,36.069130,2731 MURFREESBORO PIKE,True
20222,267126,2019-11-01,Trash - Curbside/Alley Missed Pickup,ENTIRE ALLEY,37206,METRO,9508,-86.741242,36.169482,1621 LONG AVE,True
20223,267130,2019-11-01,Trash - Curbside/Alley Missed Pickup,MISSED SEVERAL,37214,RED RIVER,1502,-86.671647,36.185643,2943 WINDEMERE CIR,True
20224,267134,2019-11-01,Trash - Curbside/Alley Missed Pickup,CALLER STATED TRASH WAS MISSED & WERE ONLY PIC...,37013,RED RIVER,4502,-86.620025,36.054637,3325 MURFREESBORO PIKE,True
